In [3]:
import cv2
import pandas as pd
import numpy as np
from pathlib import Path

import json


# Vidéo 1

In [ ]:
VIDEO_PATH = "C:\\Users\\yanis\\Downloads\\yanisVsCode\\APP\\AcquisitionsEyeTracker\\sujet1_f-42e0d11a\\e0b2c246_0.0-138.011.mp4"              # chemin vers ta vidéo POV
GAZE_CSV = "C:\\Users\\yanis\\Downloads\\yanisVsCode\\APP\\AcquisitionsEyeTracker\\sujet1_f-42e0d11a\\gaze.csv"                      # chemin vers ton fichier CSV de données de regard
WORLD_TS_CSV = "C:\\Users\\yanis\\Downloads\\yanisVsCode\\APP\\AcquisitionsEyeTracker\\sujet1_f-42e0d11a\\world_timestamps.csv"  # chemin vers ton fichier CSV de timestamps mondiaux
OUTPUT_VIDEO_PATH = "world_with_gaze_Sujet1.mp4"

# Charger paramètres caméra, ça nous sera bien utile pour plus tard !
with open("C:\\Users\\yanis\\Downloads\\yanisVsCode\\APP\\AcquisitionsEyeTracker\\sujet1_f-42e0d11a\\scene_camera.json", "r") as f:
    cam = json.load(f)

K = np.array(cam["camera_matrix"])
dist = np.array(cam["distortion_coefficients"])


# Vidéo 2

In [ ]:
VIDEO_PATH = "C:\\Users\\yanis\\Downloads\\yanisVsCode\\APP\\AcquisitionsEyeTracker\\sujet2_f-835bf855\\b7bd6c34_0.0-271.583.mp4"              # chemin vers ta vidéo POV
GAZE_CSV = "C:\\Users\\yanis\\Downloads\\yanisVsCode\\APP\\AcquisitionsEyeTracker\\sujet2_f-835bf855\\gaze.csv"                      # chemin vers ton fichier CSV de données de regard
WORLD_TS_CSV = "C:\\Users\\yanis\\Downloads\\yanisVsCode\\APP\\AcquisitionsEyeTracker\\sujet2_f-835bf855\\world_timestamps.csv"  # chemin vers ton fichier CSV de timestamps mondiaux
OUTPUT_VIDEO_PATH = "world_with_gaze_Sujet2.mp4"

# Charger paramètres caméra, plus tard on le re-utilisera d'ailleurs
with open("C:\\Users\\yanis\\Downloads\\yanisVsCode\\APP\\AcquisitionsEyeTracker\\sujet2_f-835bf855\\scene_camera.json", "r") as f:
    cam = json.load(f)

K = np.array(cam["camera_matrix"])
dist = np.array(cam["distortion_coefficients"])

# Vidéo 3

In [6]:
VIDEO_PATH = "C:\\Users\\yanis\\Downloads\\yanisVsCode\\APP\\AcquisitionsEyeTracker\\sujet3_m-84ce1158\\422f10f2_0.0-247.734.mp4"
GAZE_CSV = "C:\\Users\\yanis\\Downloads\\yanisVsCode\\APP\\AcquisitionsEyeTracker\\sujet3_m-84ce1158\\gaze.csv"
WORLD_TS_CSV = "C:\\Users\\yanis\\Downloads\\yanisVsCode\\APP\\AcquisitionsEyeTracker\\sujet3_m-84ce1158\\world_timestamps.csv"
OUTPUT_VIDEO_PATH = "world_with_gaze_Sujet3.mp4"


# Vidéo 4

In [10]:
VIDEO_PATH = "C:\\Users\\yanis\\Downloads\\yanisVsCode\\APP\\AcquisitionsEyeTracker\\sujet4_m-fee537df\\2fb8301a_0.0-71.632.mp4"
GAZE_CSV = "C:\\Users\\yanis\\Downloads\\yanisVsCode\\APP\\AcquisitionsEyeTracker\\sujet4_m-fee537df\\gaze.csv"
WORLD_TS_CSV = "C:\\Users\\yanis\\Downloads\\yanisVsCode\\APP\\AcquisitionsEyeTracker\\sujet4_m-fee537df\\world_timestamps.csv"
OUTPUT_VIDEO_PATH = "world_with_gaze_Sujet4.mp4"

# Vidéo 5

In [14]:
VIDEO_PATH = "C:\\Users\\yanis\\Downloads\\yanisVsCode\\APP\\AcquisitionsEyeTracker\\sujet5_m-671cf44e\\585d8df7_0.0-229.268.mp4"
GAZE_CSV = "C:\\Users\\yanis\\Downloads\\yanisVsCode\\APP\\AcquisitionsEyeTracker\\sujet5_m-671cf44e\\gaze.csv"
WORLD_TS_CSV = "C:\\Users\\yanis\\Downloads\\yanisVsCode\\APP\\AcquisitionsEyeTracker\\sujet5_m-671cf44e\\world_timestamps.csv"
OUTPUT_VIDEO_PATH = "world_with_gaze_Sujet5.mp4"

# Vidéo 6

In [18]:
VIDEO_PATH = "C:\\Users\\yanis\\Downloads\\yanisVsCode\\APP\\AcquisitionsEyeTracker\\sujet6_m-0b355b51\\429d311a_0.0-267.743.mp4"
GAZE_CSV = "C:\\Users\\yanis\\Downloads\\yanisVsCode\\APP\\AcquisitionsEyeTracker\\sujet6_m-0b355b51\\gaze.csv"
WORLD_TS_CSV = "C:\\Users\\yanis\\Downloads\\yanisVsCode\\APP\\AcquisitionsEyeTracker\\sujet6_m-0b355b51\\world_timestamps.csv"
OUTPUT_VIDEO_PATH = "world_with_gaze_Sujet6.mp4"

### Analyse technique du fonctionnement du programme

Le script s’organise en deux grandes étapes :  
1) *Synchroniser le regard avec chaque frame de la vidéo*  
2) *Générer une nouvelle vidéo avec le point de regard superposé*

---

### 1. Préparation et synchronisation temporelle

* On charge les données de regard (`gaze.csv`) et les timestamps vidéo (`world_timestamps.csv`).  
* On ne conserve que les colonnes nécessaires (timestamp + coordonnées du regard) et on trie tout chronologiquement.  
* On calcule la *période moyenne entre deux frames vidéo* pour estimer le rythme d’acquisition.  
* À partir de cette période, on définit une *tolérance temporelle* :  
  * elle fixe jusqu’à quel écart temporel un point de regard est encore acceptable pour une frame.  
* `merge_asof` orchestre alors la *fusion temporelle* :  
  * pour chaque frame, il récupère **l’échantillon de regard le plus proche dans le temps**,  
  * mais uniquement s’il se situe dans la tolérance définie.

→ Résultat : une table alignée où chaque frame vidéo possède soit une coordonnée de regard fiable, soit `NaN`.

---

### 2. Lecture vidéo et overlay du point de regard

* On ouvre la vidéo en lecture et on récupère ses propriétés (fps, largeur, hauteur).  
* On initialise un `VideoWriter` pour produire la vidéo annotée.  
* Dans une boucle frame par frame :
  * on lit l’image,
  * on extrait la coordonnée de regard associée,
  * si elle existe (pas de `NaN`), on :
    * “clamp” la coordonnée dans l’image,
    * **dessine un cercle rouge** au bon emplacement.
  * On écrit ensuite la frame annotée dans la vidéo de sortie.

* À la fin, on appelle `cap.release()` et `out.release()` pour fermer correctement les flux vidéo.

---

### Intention globale

*Synchroniser proprement deux sources temporelles (regard + vidéo), puis produire une vidéo annotée qui visualise, à chaque instant, la position exacte du regard dans la scène.*  
Cette orchestration prépare le terrain pour toutes les étapes suivantes : segmentation d’affiches, homographies, heatmaps, etc.


In [9]:
gaze = pd.read_csv(GAZE_CSV) # C'est les données de regard issus de la caméra interne de l'eye-tracker
world_ts = pd.read_csv(WORLD_TS_CSV) # C'est les timestamps de chaque frame de la vidéo POV 

#Ici on va affichier les heads de gaze et world_ts 
print(gaze.head())
print(world_ts.head())

gaze = gaze[["timestamp [ns]", "gaze x [px]", "gaze y [px]"]].copy()
world_ts = world_ts[["timestamp [ns]"]].copy()

print(gaze.head())
print(world_ts.head())


frame_times = world_ts["timestamp [ns]"].values # Ici on va se caler le tableau de tous les timestamps vidéo.
frame_intervals = np.diff(frame_times) # Là calcule la différence entre chaque timestamp → on a les intervalles entre frames.
median_dt = np.median(frame_intervals)  # en nanosecondes
# tolérance = la moitié d'une période de frame
tolerance_ns = int(median_dt/2)

# merge_asof associe à chaque timestamp de frame l'échantillon de gaze le plus proche
merged = pd.merge_asof(
    world_ts,
    gaze,
    on="timestamp [ns]",
    direction="nearest",
    tolerance=tolerance_ns
)
# merged contient maintenant pour chaque frame :
# - timestamp [ns]
# - gaze x [px]
# - gaze y [px] (NaN si aucun gaze assez proche dans la tolérance)
print(merged.head())


# Lecture de la vidéo + overlay des points
print("Ouverture de la vidéo...")
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise FileNotFoundError(f"Impossible d'ouvrir la vidéo: {VIDEO_PATH}")

# Récupération infos vidéo
fps = cap.get(cv2.CAP_PROP_FPS) # ici fps représente le nombre de frames par seconde de la vidéo
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # ici width représente la largeur de la vidéo en pixels
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # ici height représente la hauteur de la vidéo en pixels

print(f"Vidéo : {len(world_ts)} frames, {fps:.2f} fps, taille = {width}x{height}")


# Préparation de la vidéo de sortie
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (width, height))

print("Début du traitement des frames...")
for i in range(len(world_ts)):
    ret, frame = cap.read()
    if not ret:
        print(f"Fin de lecture vidéo à la frame {i}")
        break

    row = merged.iloc[i]
    x = row["gaze x [px]"]
    y = row["gaze y [px]"]

    # Si on a bien un point de regard pour cette frame
    if not (np.isnan(x) or np.isnan(y)):
        # Conversion en int, clamp pour éviter de sortir de l'image
        px = int(np.clip(x, 0, width - 1))
        py = int(np.clip(y, 0, height - 1))

        # Dessiner un cercle rouge (BGR = (0, 0, 255))
        cv2.circle(frame, (px, py), radius=10, color=(0, 0, 255), thickness=-1)

    out.write(frame)

cap.release()
out.release()
print(f"Vidéo avec regard enregistrée dans: {OUTPUT_VIDEO_PATH}")

                             section id                          recording id  \
0  e0b2c246-195c-41f2-a3eb-f387f9f86d9a  42e0d11a-5676-4755-bf47-0bbe4d84aceb   
1  e0b2c246-195c-41f2-a3eb-f387f9f86d9a  42e0d11a-5676-4755-bf47-0bbe4d84aceb   
2  e0b2c246-195c-41f2-a3eb-f387f9f86d9a  42e0d11a-5676-4755-bf47-0bbe4d84aceb   
3  e0b2c246-195c-41f2-a3eb-f387f9f86d9a  42e0d11a-5676-4755-bf47-0bbe4d84aceb   
4  e0b2c246-195c-41f2-a3eb-f387f9f86d9a  42e0d11a-5676-4755-bf47-0bbe4d84aceb   

        timestamp [ns]  gaze x [px]  gaze y [px]  worn  fixation id  blink id  \
0  1763456864828816499      435.227      576.743     1          1.0       NaN   
1  1763456864833816499      445.815      577.747     1          1.0       NaN   
2  1763456864838816499      447.881      574.315     1          1.0       NaN   
3  1763456864843816499      449.868      571.571     1          1.0       NaN   
4  1763456864848816499      453.487      572.783     1          1.0       NaN   

   azimuth [deg]  elevatio

### Objectif général du script

Ce module applique une **correction de distorsion** à une vidéo à partir des paramètres de calibration contenus dans un fichier JSON.  
Il constitue une étape préalable indispensable avant :

- la détection des posters,
- le recalage géométrique par homographie,
- la projection du regard dans le repère scène,
- la génération des heatmaps.

Corriger la distorsion **avant tout traitement géométrique** garantit une scène cohérente, améliore la stabilité des homographies et supprime les déformations optiques du fish-eye.

---

### 1. Chargement de la calibration (`load_calibration()`)

Cette fonction :

- ouvre le fichier JSON généré par l’eye-tracker,
- extrait la **matrice intrinsèque K** (fx, fy, cx, cy),
- récupère les **coefficients de distorsion** (radiaux + tangentiel),
- convertit l’ensemble en `numpy.float32` afin d’être compatible avec OpenCV.

Ce chargement constitue la base nécessaire au modèle de projection pinhole + distorsion utilisé dans OpenCV.  
Le vecteur de distorsion peut avoir **4, 5 ou 8 coefficients** — le script les conserve tous pour un maximum de fidélité.

En sortie, la fonction renvoie :

- **K** : matrice 3×3 de la caméra  
- **dist** : vecteur (n×1) des coefficients de distorsion

---

### 2. Ouverture de la vidéo et lecture des propriétés

Dans `undistort_video()`, le script :

- ouvre la vidéo d’entrée,
- lit ses propriétés essentielles :
  - résolution `(width, height)`
  - nombre total de frames
  - FPS

Ces informations servent à :

- dimensionner correctement la matrice optimisée,
- initialiser la vidéo de sortie,
- afficher une progression propre pendant le traitement.

Un contrôle d’erreur est appliqué dès l’ouverture pour éviter un traitement silencieux sur un fichier inexistant.

---

### 3. Calcul de la matrice caméra optimisée (`getOptimalNewCameraMatrix`)

OpenCV génère une nouvelle matrice intrinsèque tenant compte de la distorsion, qui permet :

- de **réduire les bords noirs**,  
- ou de **conserver le champ de vision complet**.

Le paramètre **ALPHA** gouverne ce compromis :

- **0.0 → crop maximum**, moins de bords noirs  
- **1.0 → champ complet**, mais davantage de zones noires

La fonction retourne aussi un **ROI** utile si l’on souhaite recadrer strictement la zone valide après correction.

---

### 4. Pré-calcul des cartes de remapping (`initUndistortRectifyMap`)

Cette étape prépare deux cartes :

- `map1` : coordonnées x corrigées  
- `map2` : coordonnées y corrigées  

Elles permettent d’appliquer la correction optique avec un simple appel à `cv2.remap()` sur chaque frame.  
L’utilisation de `cv2.CV_16SC2` permet d’optimiser la mémoire et la vitesse d’exécution.

Ce pré-calcul évite de recalculer la géométrie pour chaque image → **gain de performance majeur**.

---

### 5. Traitement frame-par-frame

Pour chaque image :

1. Lecture de la frame brute  
2. Correction via : cv2.remap(frame, map1, map2, interpolation=cv2.INTER_LINEAR)
3. Écriture dans la vidéo corrigée

Un log est affiché toutes les **50 frames** pour assurer un suivi propre de la progression.

Une option commentée permet de croper strictement à la zone valide fournie par le ROI.

---

### 6. Finalisation

À la fin du traitement :

- la vidéo d’entrée est libérée,  
- la vidéo de sortie est fermée proprement,  
- un message indique l’emplacement de la vidéo corrigée.

Le résultat est une vidéo **rectifiée**, de même taille et même fps que l’originale, prête pour les étapes suivantes :  
ORB → homographie → regard → heatmaps.

---

### Résumé du pipeline

1. Charger calibration → **K, dist**  
2. Ouvrir vidéo → **fps, taille, frame_count**  
3. Optimiser la matrice caméra → **new_K, ROI**  
4. Générer les cartes → **map1, map2**  
5. Corriger chaque frame → **remap()**  
6. Exporter la vidéo corrigée  

Pipeline fiable, performant, et totalement compatible avec le reste du projet.




In [5]:
import cv2
import json
import numpy as np
from pathlib import Path

# -----------------------------
# PARAMÈTRES À ADAPTER
# -----------------------------
CALIB_PATH   = Path(r"C:\Users\yanis\Downloads\yanisVsCode\APP\AcquisitionsEyeTracker\sujet5_m-671cf44e\scene_camera.json")  # fichier de calibration
INPUT_VIDEO  = r"C:\Users\yanis\Downloads\yanisVsCode\APP\Video_Eye_Tracking\world_with_gaze_Sujet5.mp4"              # ta vidéo d'entrée (avec ou sans gaze)
OUTPUT_VIDEO = "world_undistorted_Sujet5.mp4"     # vidéo corrigée
ALPHA        = 0.0                         # 0 = crop maximum des bords noirs, 1 = garde tout


def load_calibration(calib_path: Path):
    """Charge la matrice caméra K et les coefficients de distorsion depuis le JSON."""
    with open(calib_path, "r") as f:
        data = json.load(f)

    K = np.array(data["camera_matrix"], dtype=np.float32)
    # OpenCV accepte un vecteur 4, 5 ou 8 pour la distorsion -> on garde tout
    dist = np.array(data["distortion_coefficients"], dtype=np.float32).reshape(-1, 1)

    print("Matrice caméra K :\n", K)
    print("Coefficients de distorsion :", dist.ravel())
    return K, dist


def undistort_video(input_path: str, output_path: str, calib_path: Path, alpha: float = 0.0):
    # 1. Charger la calibration
    K, dist = load_calibration(calib_path)

    # 2. Ouvrir la vidéo
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        raise FileNotFoundError(f"Impossible d'ouvrir la vidéo : {input_path}")

    fps    = cap.get(cv2.CAP_PROP_FPS)
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"Vidéo d'entrée : {frame_count} frames, {fps:.2f} fps, taille = {width}x{height}")

    # 3. Calcul de la nouvelle matrice caméra optimisée
    #    ALPHA contrôle le compromis :
    #    - 0.0 : moins de bords noirs, mais un léger crop
    #    - 1.0 : conserve tout le champ de vision (plus de bords noirs)
    new_K, roi = cv2.getOptimalNewCameraMatrix(K, dist, (width, height), alpha, (width, height))
    print("Matrice caméra optimisée new_K :\n", new_K)
    print("ROI :", roi)

    # 4. Pré-calcul des cartes de remapping (plus efficace que undistort frame par frame)
    map1, map2 = cv2.initUndistortRectifyMap(
        K, dist, None, new_K, (width, height), cv2.CV_16SC2
    )

    # 5. Préparation de la vidéo de sortie (même taille et mêmes fps)
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 6. Appliquer la correction de distorsion
        undistorted = cv2.remap(frame, map1, map2, interpolation=cv2.INTER_LINEAR)

        # Si tu veux croper strictement à la zone valide, décommente :
        # x, y, w, h = roi
        # undistorted = undistorted[y:y+h, x:x+w]

        out.write(undistorted)
        frame_idx += 1

        if frame_idx % 50 == 0:
            print(f"{frame_idx}/{frame_count} frames traitées...")

    cap.release()
    out.release()
    print(f"Vidéo corrigée enregistrée dans : {output_path}")


if __name__ == "__main__":
    undistort_video(INPUT_VIDEO, OUTPUT_VIDEO, CALIB_PATH, alpha=ALPHA)


Matrice caméra K :
 [[884.9203   0.     797.3327]
 [  0.     884.8444 590.4394]
 [  0.       0.       1.    ]]
Coefficients de distorsion : [-0.1284729   0.10799497 -0.00076467  0.00041167  0.00231511  0.1688897
  0.05152452  0.02927154]
Vidéo d'entrée : 6838 frames, 29.82 fps, taille = 1600x1200
Matrice caméra optimisée new_K :
 [[628.91     0.     798.5339]
 [  0.     749.0824 584.9927]
 [  0.       0.       1.    ]]
ROI : (0, 0, 1599, 1199)
50/6838 frames traitées...
100/6838 frames traitées...
150/6838 frames traitées...
200/6838 frames traitées...
250/6838 frames traitées...
300/6838 frames traitées...
350/6838 frames traitées...
400/6838 frames traitées...
450/6838 frames traitées...
500/6838 frames traitées...
550/6838 frames traitées...
600/6838 frames traitées...
650/6838 frames traitées...
700/6838 frames traitées...
750/6838 frames traitées...
800/6838 frames traitées...
850/6838 frames traitées...
900/6838 frames traitées...
950/6838 frames traitées...
1000/6838 frames trai

### Objectif du script

Ce module détecte automatiquement des **affiches** dans une vidéo grâce à ORB, au matching de descripteurs et à l’estimation d’homographies robustes.  
Il sert de base pour les étapes suivantes : localisation des posters, association avec le regard, et génération de heatmaps.

---

### 1. Chargement des affiches

La fonction `load_posters()` :

* parcourt `POSTERS_DIR` et charge chaque image (.png/.jpg),
* réduit sa taille si nécessaire (`MAX_POSTER_SIZE`) pour accélérer le matching,
* extrait des keypoints + descripteurs ORB,
* définit les **4 coins** du poster dans son repère image pour permettre sa projection dans la vidéo.

Chaque affiche est stockée avec :

* `name`,
* `kp` / `des` (features ORB),
* `corners` (rectangle de référence),
* `image` (pour debug éventuel).

---

### 2. Détection dans une frame vidéo

`detect_posters_in_frame(frame, posters, orb, bf)` effectue :

1. Extraction ORB de la frame.
2. Pour chaque poster :
   * Matching KNN des descripteurs (poster → frame),
   * Application du **Lowe ratio test** (`RATIO_TEST`),
   * Rejet si peu de bons matches (`MIN_GOOD_MATCHES`),
   * Calcul d’une **homographie** via `cv2.findHomography(..., RANSAC)`,
   * Vérification du nombre d’inliers (`MIN_INLIERS`),
   * Projection des coins du poster dans la frame via `cv2.perspectiveTransform`.

Chaque détection retournée contient :

* `name`,
* `corners` projetés,
* `inliers`,
* `good_matches`.

---

### 3. Dessin des détections

`draw_detections()` :

* trace un polygone vert autour du poster détecté,
* affiche un label : `NomAffiche (nb_inliers)`.

Cela permet de visualiser rapidement la qualité et la fiabilité de la détection.

---

### 4. Orchestration générale (`main()`)

La fonction principale :

1. Initialise ORB + BFMatcher.
2. Charge toutes les affiches (features pré-calculées).
3. Ouvre la vidéo (`VIDEO_PATH`) et lit ses propriétés (fps, taille).
4. Crée une vidéo de sortie (`OUTPUT_VIDEO`).
5. Pour chaque frame :
   * détecte les affiches,
   * dessine les cadres,
   * écrit la frame annotée.
6. Libère proprement les ressources.

Le script affiche un log toutes les **50 frames** pour suivre la progression.


In [2]:
import cv2
import numpy as np
from pathlib import Path

# ---------------------------------------------------------
# PARAMÈTRES GÉNÉRAUX
# ---------------------------------------------------------
# → Mets ici ta vidéo (idéalement la version undistordue)
VIDEO_PATH   = r"C:\Users\yanis\Downloads\yanisVsCode\APP\Video_Eye_Tracking\VideoSansDistortion\world_undistorted_Sujet3.mp4"
# → Dossier contenant les images des affiches (PNG ou JPG)
POSTERS_DIR  = Path(r"C:\Users\yanis\Downloads\yanisVsCode\APP\posters")
# → Vidéo de sortie avec les cadres autour des affiches détectées
OUTPUT_VIDEO = r"C:\Users\yanis\Downloads\yanisVsCode\APP\Video_Eye_Tracking\ORB_testV1_Sujet3.mp4"
# ---------------------------------------------------------
# PARAMÈTRES ORB / MATCHING / HOMOGRAPHIE
# ---------------------------------------------------------
MAX_POSTER_SIZE   = 1000   # taille max (en pixels) pour le plus grand côté de l'affiche
ORB_NFEATURES     = 2000   # nombre max de keypoints ORB
RATIO_TEST        = 0.75   # Lowe ratio test
MIN_GOOD_MATCHES  = 25     # nombre mini de bons matches avant homographie
MIN_INLIERS       = 25     # nombre mini d'inliers RANSAC pour valider une affiche



# ---------------------------------------------------------
# CHARGEMENT DES AFFICHES ET EXTRACTION DES FEATURES
# ---------------------------------------------------------
def load_posters(orb, posters_dir: Path):
    """
    Charge toutes les affiches dans posters_dir, les redimensionne
    si besoin, extrait les keypoints ORB et prépare les coins du rectangle
    de référence pour la projection via homographie.
    """
    posters = []

    if not posters_dir.exists():
        raise FileNotFoundError(f"Dossier d'affiches introuvable : {posters_dir}")

    for img_path in posters_dir.glob("*.*"):
        if img_path.suffix.lower() not in [".png", ".jpg", ".jpeg"]:
            continue

        img = cv2.imread(str(img_path), cv2.IMREAD_COLOR)
        if img is None:
            print(f"⚠ Impossible de lire {img_path}")
            continue

        # Redimensionnement éventuel pour limiter la résolution
        h, w = img.shape[:2]
        scale = min(MAX_POSTER_SIZE / max(w, h), 1.0)  # <= 1.0 -> jamais agrandir
        if scale < 1.0:
            new_w = int(w * scale)
            new_h = int(h * scale)
            img = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
            print(f"{img_path.stem} redimensionnée à {new_w}x{new_h}")
        else:
            print(f"{img_path.stem} gardée à taille originale {w}x{h}")

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        kp, des = orb.detectAndCompute(gray, None)
        h, w = gray.shape

        # Coins du poster dans son repère image (rectangle)
        corners = np.array(
            [[0,   0],
             [w-1, 0],
             [w-1, h-1],
             [0,   h-1]],
            dtype=np.float32
        ).reshape(-1, 1, 2)

        posters.append({
            "name": img_path.stem,
            "image": img,
            "kp": kp,
            "des": des,
            "corners": corners
        })
        print(f"Affiche chargée : {img_path.stem}, {len(kp)} keypoints")

    return posters


# ---------------------------------------------------------
# DÉTECTION D'UNE OU PLUSIEURS AFFICHES DANS UNE FRAME
# ---------------------------------------------------------
def detect_posters_in_frame(frame, posters, orb, bf):
    """
    Retourne une liste de détections pour la frame :
    [ { "name": str, "corners": corners_trans, "inliers": int, "good_matches": int }, ... ]
    """
    detections = []

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    kp_frame, des_frame = orb.detectAndCompute(gray_frame, None)

    if des_frame is None or len(kp_frame) == 0:
        return detections

    for poster in posters:
        des_p = poster["des"]
        kp_p  = poster["kp"]
        if des_p is None or len(kp_p) == 0:
            continue

        # Matching KNN des descripteurs de l'affiche vers ceux de la frame
        matches = bf.knnMatch(des_p, des_frame, k=2)

        # Lowe ratio test
        good = []
        for m, n in matches:
            if m.distance < RATIO_TEST * n.distance:
                good.append(m)

        if len(good) < MIN_GOOD_MATCHES:
            # Pas assez de correspondances pour tenter une homographie
            continue

        # Points source (affiche) et destination (frame)
        src_pts = np.float32(
            [kp_p[m.queryIdx].pt for m in good]
        ).reshape(-1, 1, 2)

        dst_pts = np.float32(
            [kp_frame[m.trainIdx].pt for m in good]
        ).reshape(-1, 1, 2)

        # Homographie robuste via RANSAC
        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        if H is None:
            continue

        mask = mask.ravel().tolist()
        inliers = sum(mask)

        if inliers < MIN_INLIERS:
            continue

        # Projection des coins du poster dans la frame
        corners_trans = cv2.perspectiveTransform(poster["corners"], H)

        detections.append({
            "name": poster["name"],
            "corners": corners_trans,
            "inliers": inliers,
            "good_matches": len(good)
        })

    return detections


# ---------------------------------------------------------
# DESSIN DES RÉSULTATS SUR LA FRAME
# ---------------------------------------------------------
def draw_detections(frame, detections):
    """
    Dessine les cadres + labels des affiches détectées sur la frame.
    """
    for det in detections:
        pts = det["corners"].astype(int)

        # tracer le contour du poster
        cv2.polylines(frame, [pts], isClosed=True, color=(0, 255, 0), thickness=3)

        # texte (nom + nb d'inliers)
        x, y = pts[0, 0, 0], pts[0, 0, 1]
        label = f"{det['name']} ({det['inliers']} inliers)"
        cv2.putText(frame, label, (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)

    return frame


# ---------------------------------------------------------
# ORCHESTRATION GLOBALE
# ---------------------------------------------------------
def main():
    # ORB + BFMatcher
    orb = cv2.ORB_create(nfeatures=ORB_NFEATURES)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)

    # 1. Charger les affiches
    posters = load_posters(orb, POSTERS_DIR)
    if not posters:
        raise RuntimeError("Aucune affiche chargée, vérifie le dossier POSTERS_DIR.")

    # 2. Ouvrir la vidéo
    cap = cv2.VideoCapture(VIDEO_PATH)
    if not cap.isOpened():
        raise FileNotFoundError(f"Impossible d'ouvrir la vidéo {VIDEO_PATH}")

    fps         = cap.get(cv2.CAP_PROP_FPS)
    width       = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height      = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"Vidéo : {frame_count} frames, {fps:.2f} fps, taille = {width}x{height}")

    # 3. Préparer la vidéo de sortie
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, (width, height))

    # 4. Boucle frame par frame
    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        detections = detect_posters_in_frame(frame, posters, orb, bf)
        frame_out = draw_detections(frame, detections)
        out.write(frame_out)

        frame_idx += 1
        if frame_idx % 50 == 0:
            print(f"{frame_idx}/{frame_count} frames traitées...")

    cap.release()
    out.release()
    print(f"✅ Vidéo avec détection des affiches : {OUTPUT_VIDEO}")


if __name__ == "__main__":
    main()


Anning redimensionnée à 666x1000
Affiche chargée : Anning, 2000 keypoints
Barres redimensionnée à 666x1000
Affiche chargée : Barres, 2000 keypoints
Bell redimensionnée à 666x1000
Affiche chargée : Bell, 2000 keypoints
Bunten-Berry redimensionnée à 666x1000
Affiche chargée : Bunten-Berry, 2000 keypoints
Franklin redimensionnée à 666x1000
Affiche chargée : Franklin, 2000 keypoints
Gautier redimensionnée à 666x1000
Affiche chargée : Gautier, 2000 keypoints
Johnson redimensionnée à 666x1000
Affiche chargée : Johnson, 2000 keypoints
Noether redimensionnée à 666x1000
Affiche chargée : Noether, 2000 keypoints
Vidéo : 7388 frames, 29.82 fps, taille = 1600x1200
50/7388 frames traitées...
100/7388 frames traitées...
150/7388 frames traitées...
200/7388 frames traitées...
250/7388 frames traitées...
300/7388 frames traitées...
350/7388 frames traitées...
400/7388 frames traitées...
450/7388 frames traitées...
500/7388 frames traitées...
550/7388 frames traitées...
600/7388 frames traitées...
650/

KeyboardInterrupt: 

### Objectif général du module

Ce script réalise l’ensemble du pipeline **détection d’affiches → association avec le regard → génération de heatmaps**, en travaillant directement à partir :

- des vidéos corrigées de l’eye-tracker,
- des CSV gaze + timestamps monde,
- des images d’affiches,
- et des matrices caméra/distorsion pour corriger les points de regard.

L’objectif final est de produire **une heatmap par affiche**, représentant la distribution spatiale du regard projeté dans le repère de l’affiche (homographie inverse).

---

### 1. Paramètres globaux & calibration caméra

La première section déclare :

- les chemins vers la vidéo, les CSV (gaze + world timestamps), et le dossier des affiches,
- les hyperparamètres ORB / matching / homographie,
- la résolution des heatmaps,
- les paramètres intrinsèques caméra (fx, fy, cx, cy) et la distorsion (8 coefficients de l’eye-tracker).

Ces éléments servent à :

- normaliser les points de regard via `cv2.undistortPoints`,
- assurer la cohérence géométrique entre la vidéo, les projections et les homographies,
- dimensionner correctement les heatmaps (downscale configurable).

On affiche aussi K et le vecteur de distorsion pour vérifier que la calibration est correctement chargée.

---

### 2. Fusion regard ⇄ timestamps monde (avec undistortion)

`build_merged_gaze_world()` fusionne les données de regard avec la timeline vidéo grâce à `merge_asof`.

Pipeline :

1. **Chargement des CSV gaze + world timestamps**  
   On extrait uniquement les colonnes nécessaires.

2. **Correction optique du regard**  
   On applique : cv2.undistortPoints(pts, K, dist, P=new_camera_matrix)
   
→ conversion des coordonnées distordues en coordonnées **undistordues dans le même repère que la vidéo corrigée**.

3. **Tolérance temporelle automatique**  
On estime la période médiane entre frames vidéo pour définir la tolérance d’alignement gaze ↔ frame.

4. **Fusion finalisée**  
Chaque frame vidéo possède désormais une valeur gaze (x, y) interpolée temporellement.

Cette fusion garantit que chaque frame peut être traitée avec un regard cohérent, même en cas de petites variations de timestamp.

---

### 3. Suppression des doublons via NMS

L’algorithme NMS maison permet d’éviter d’avoir plusieurs détections du même poster dans une frame.

Détails :

- `bbox_from_corners()` convertit les coins transformés en boîte axis-aligned.
- `iou()` calcule l’intersection sur union entre deux boîtes.
- `suppress_overlaps()` :
- trie les détections par score (nb d’inliers),
- supprime les doublons selon un `iou_thresh`,
- ne conserve que les posters les plus fiables.

Le NMS garantit que chaque poster est représenté **une seule fois par frame**, ce qui rend la heatmap plus propre.

---

### 4. Chargement des affiches & extraction ORB

`load_posters()` :

1. Charge chaque image d’affiche depuis `POSTERS_DIR`.
2. Réduit sa taille si nécessaire (`MAX_POSTER_SIZE`) pour accélérer ORB.
3. Convertit en niveau de gris, extrait :
- les keypoints,
- les descripteurs ORB.
4. Définit les **coins de référence** du poster :  [0,0], [w-1,0], [w-1,h-1], [0,h-1]
5. Stocke chaque poster dans une structure contenant :
- `name`
- `img`
- `kp` / `des`
- `corners`
- `size`

Les posters ainsi pré-traités peuvent être directement utilisés pour le matching dans chaque frame vidéo.

---

### 5. Détection des posters dans une frame vidéo

`detect_posters_in_frame()` effectue :

#### 1) Extraction des features de la frame
- conversion en gris,
- ORB.detectAndCompute.

#### 2) Matching KNN poster → frame
On utilise un BFMatcher Hamming :
- ratio test Lowe (`RATIO_TEST`),
- filtrage des mauvais matches.

#### 3) Homographie RANSAC
Si assez de matches :
- estimation H = `cv2.findHomography`,
- comptage des inliers,
- rejet si `< MIN_INLIERS`.

#### 4) Projection du rectangle du poster
On transforme les coins avec : cv2.perspectiveTransform(poster["corners"], H)

#### 5) Application du NMS
On retourne une liste nettoyée de détections valides.

Chaque détection contient :
- nom du poster,
- coins projetés,
- nb d’inliers,
- nb de bons matches,
- matrice H,
- taille du poster.

---

### 6. Initialisation & mise à jour des heatmaps

#### `init_heatmaps()`
Crée une heatmap vide (float32) pour chaque poster, avec une résolution réduite (`HEATMAP_DOWNSCALE`).

#### `update_heatmaps()`
Pour chaque detection :

1. Test si le regard se situe **dans le quadrilatère projeté** (pointPolygonTest).
2. Conversion du regard frame → poster via l’homographie inverse : uv = H_inv * [gx, gy]

3. Normalisation dans le repère de la heatmap (u/w, v/h).
4. Incrément de la cellule correspondante.

Ce mécanisme associe chaque regard à sa position exacte dans le poster, indépendamment de la perspective.

---

### 7. Génération & sauvegarde des heatmaps

`save_heatmaps()` :

1. Lisse la heatmap via un Gaussian blur.
2. Normalise et convertit en image 8 bits.
3. Colorise (`COLORMAP_JET`) puis upscale à la taille de l’affiche.
4. Fusionne l’affiche + heatmap via alpha blending.
5. Sauvegarde le fichier final sous forme PNG.

Chaque poster reçoit son image de chaleur finale.

---

### 8. Pipeline principal (`main()`)

La boucle globale :

1. Construit le merged gaze/world.
2. Initialise ORB + matcher.
3. Charge les affiches.
4. Initialise les heatmaps.
5. Parcourt la vidéo frame par frame :
- détecte les posters,
- récupérer gaze_x, gaze_y alignés,
- met à jour la heatmap correspondante.
6. Affiche un log toutes les 50 frames.
7. Enregistre toutes les heatmaps à la fin.

Aucune vidéo n’est générée : seule la **carte thermique finale** pour chaque poster est produite.

---

### Résumé global du pipeline

1. Correction du regard (undistortion)  
2. Fusion temporelle gaze ↔ vidéo  
3. Extraction ORB affiches + frame  
4. Matching + ratio test  
5. Homographie RANSAC + projection  
6. NMS pour éviter les doublons  
7. Homographie inverse pour projeter le regard  
8. Génération des heatmaps lissées  
9. Sauvegarde finale poster + heatmap

Pipeline complet, propre et robuste pour analyser très précisément la distribution du regard sur chaque affiche.








In [6]:
import cv2
import numpy as np
import pandas as pd
from pathlib import Path

# ============================================================
# PATHS À ADAPTER
# ============================================================

VIDEO_PATH = r"C:\Users\yanis\Downloads\yanisVsCode\APP\Video_Eye_Tracking\ORB_testV3_Sujet4.mp4"
GAZE_CSV = "C:\\Users\\yanis\\Downloads\\yanisVsCode\\APP\\AcquisitionsEyeTracker\\sujet4_m-fee537df\\gaze.csv"
WORLD_TS_CSV = "C:\\Users\\yanis\\Downloads\\yanisVsCode\\APP\\AcquisitionsEyeTracker\\sujet4_m-fee537df\\world_timestamps.csv"
POSTERS_DIR     = Path(r"C:\Users\yanis\Downloads\yanisVsCode\APP\posters")

# résolution de la heatmap (réduction par rapport à l’affiche)
HEATMAP_DOWNSCALE   = 8
GAUSSIAN_BLUR_KSIZE = (35, 35)

# ORB / matching / homographie
MAX_POSTER_SIZE  = 1000
ORB_NFEATURES    = 2000
RATIO_TEST       = 0.70
MIN_GOOD_MATCHES = 20
MIN_INLIERS      = 20


# ============================================================
# CALIBRATION CAMÉRA  (dépendant de eye-tracker)
# ============================================================

fx = 884.9202751251051
fy = 884.8443962022465
cx = 797.3326895814895
cy = 590.4393801156965

camera_matrix = np.array([
    [fx, 0.0, cx],
    [0.0, fy, cy],
    [0.0, 0.0, 1.0]
], dtype=np.float32)

# Distorsion fournie (8 coefficients dans vos JSON)
dist_coeffs = np.array([
    -0.12847289024405859,
     0.10799496401849330,
    -0.000764670173249848,
     0.0004116699644771188,
     0.0023151069375056533,
     0.168889696495412,
     0.0515245213892395,
     0.029271541722755288
], dtype=np.float32)

# Si tu utilises la convention OpenCV (k1,k2,p1,p2,k3) tu peux prendre les 5 premiers éléments :
dist_coeffs_opencv5 = dist_coeffs[:5].copy()

# new_camera_matrix : utiliser la même si tu n'as pas recalculé newK via getOptimalNewCameraMatrix
new_camera_matrix = camera_matrix.copy()

# Affichage rapide pour vérification
print('K =\\n', camera_matrix)
print('dist (8) =', dist_coeffs)
print('dist (opencv 5) =', dist_coeffs_opencv5)

# ============================================================
# MERGE GAZE / WORLD EN TENANT COMPTE DE L’UNDISTORTION
# ============================================================

def build_merged_gaze_world():
    gaze = pd.read_csv(GAZE_CSV)
    world_ts = pd.read_csv(WORLD_TS_CSV)

    gaze = gaze[["timestamp [ns]", "gaze x [px]", "gaze y [px]"]].copy()
    world_ts = world_ts[["timestamp [ns]"]].copy()

    # undistort des points de regard (coordonnées caméra distordue -> undistordue)
    pts = gaze[["gaze x [px]", "gaze y [px]"]].values.astype(np.float32).reshape(-1, 1, 2)
    pts_undist = cv2.undistortPoints(pts, camera_matrix, dist_coeffs, P=new_camera_matrix)

    gaze["gaze x [px]"] = pts_undist[:, 0, 0]
    gaze["gaze y [px]"] = pts_undist[:, 0, 1]

    # tolérance temporelle = moitié de la période entre deux frames vidéo
    frame_times = world_ts["timestamp [ns]"].values
    frame_intervals = np.diff(frame_times)
    median_dt = np.median(frame_intervals)         # ns
    tolerance_ns = int(median_dt / 2)

    merged = pd.merge_asof(
        world_ts.sort_values("timestamp [ns]"),
        gaze.sort_values("timestamp [ns]"),
        on="timestamp [ns]",
        direction="nearest",
        tolerance=tolerance_ns
    )

    return merged


# ============================================================
# NMS POUR ÉVITER LES DOUBLONS DE CADRES
# ============================================================

def bbox_from_corners(corners: np.ndarray) -> np.ndarray:
    pts = corners.reshape(-1, 2)
    x_min = pts[:, 0].min()
    y_min = pts[:, 1].min()
    x_max = pts[:, 0].max()
    y_max = pts[:, 1].max()
    return np.array([x_min, y_min, x_max, y_max], dtype=np.float32)

def iou(b1: np.ndarray, b2: np.ndarray) -> float:
    xA = max(b1[0], b2[0])
    yA = max(b1[1], b2[1])
    xB = min(b1[2], b2[2])
    yB = min(b1[3], b2[3])

    inter_w = max(0.0, xB - xA)
    inter_h = max(0.0, yB - yA)
    inter   = inter_w * inter_h
    if inter <= 0:
        return 0.0

    area1 = (b1[2] - b1[0]) * (b1[3] - b1[1])
    area2 = (b2[2] - b2[0]) * (b2[3] - b2[1])
    union = area1 + area2 - inter
    if union <= 0:
        return 0.0
    return inter / union

def suppress_overlaps(detections, iou_thresh: float = 0.3):
    if not detections:
        return []

    for det in detections:
        det["bbox"]  = bbox_from_corners(det["corners"])
        det["score"] = det["inliers"]

    dets = sorted(detections, key=lambda d: d["score"], reverse=True)
    kept = []
    for det in dets:
        keep = True
        for k in kept:
            if iou(det["bbox"], k["bbox"]) > iou_thresh:
                keep = False
                break
        if keep:
            kept.append(det)

    for det in kept:
        det.pop("bbox", None)
        det.pop("score", None)

    return kept


# ============================================================
# CHARGEMENT DES AFFICHES + ORB
# ============================================================

def load_posters(orb, posters_dir: Path):
    posters = []
    if not posters_dir.exists():
        raise FileNotFoundError(f"Dossier d'affiches introuvable : {posters_dir}")

    for img_path in posters_dir.glob("*"):
        if img_path.suffix.lower() not in [".png", ".jpg", ".jpeg"]:
            continue

        img = cv2.imread(str(img_path), cv2.IMREAD_COLOR)
        if img is None:
            print(f"⚠️ Impossible de lire {img_path}")
            continue

        h, w = img.shape[:2]
        scale = min(MAX_POSTER_SIZE / max(w, h), 1.0)
        if scale < 1.0:
            new_w = int(w * scale)
            new_h = int(h * scale)
            img   = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
            w, h  = new_w, new_h

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        kp, des = orb.detectAndCompute(gray, None)

        corners = np.array([
            [0, 0],
            [w - 1, 0],
            [w - 1, h - 1],
            [0, h - 1]
        ], dtype=np.float32).reshape(-1, 1, 2)

        posters.append({
            "name": img_path.stem,
            "img": img,
            "kp": kp,
            "des": des,
            "corners": corners,
            "size": (w, h)
        })

        print(f"Affiche chargée : {img_path.stem} ({len(kp)} keypoints)")

    return posters


# ============================================================
# DÉTECTION DES POSTERS DANS UNE FRAME
# ============================================================

def detect_posters_in_frame(frame, posters, orb, bf):
    detections = []
    if frame is None:
        return detections

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    kp_frame, des_frame = orb.detectAndCompute(gray_frame, None)
    if des_frame is None or len(kp_frame) == 0:
        return detections

    for poster in posters:
        des_p = poster["des"]
        kp_p  = poster["kp"]
        if des_p is None or len(kp_p) == 0:
            continue

        matches = bf.knnMatch(des_p, des_frame, k=2)
        good = []
        for m, n in matches:
            if m.distance < RATIO_TEST * n.distance:
                good.append(m)

        if len(good) < MIN_GOOD_MATCHES:
            continue

        src_pts = np.float32([kp_p[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp_frame[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        if H is None:
            continue

        mask    = mask.ravel().tolist()
        inliers = sum(mask)
        if inliers < MIN_INLIERS:
            continue

        corners_trans = cv2.perspectiveTransform(poster["corners"], H)

        detections.append({
            "name": poster["name"],
            "corners": corners_trans,
            "inliers": inliers,
            "good_matches": len(good),
            "H": H,
            "size": poster["size"]
        })

    return suppress_overlaps(detections)


# ============================================================
# HEATMAPS
# ============================================================

def init_heatmaps(posters):
    heatmaps = {}
    for p in posters:
        w, h = p["size"]
        hm_w = max(1, w // HEATMAP_DOWNSCALE)
        hm_h = max(1, h // HEATMAP_DOWNSCALE)
        heatmaps[p["name"]] = np.zeros((hm_h, hm_w), dtype=np.float32)
    return heatmaps

def update_heatmaps(detections, gaze_x, gaze_y, heatmaps):
    for det in detections:
        name    = det["name"]
        corners = det["corners"].reshape(-1, 2).astype(np.float32)

        inside = cv2.pointPolygonTest(corners, (float(gaze_x), float(gaze_y)), False)
        if inside < 0:
            continue

        H_inv = np.linalg.inv(det["H"])
        p = np.array([[[gaze_x, gaze_y]]], dtype=np.float32)
        uv = cv2.perspectiveTransform(p, H_inv)[0][0]
        u, v = uv

        w_p, h_p = det["size"]
        hm = heatmaps[name]
        hm_h, hm_w = hm.shape

        ui = int((u / w_p) * hm_w)
        vi = int((v / h_p) * hm_h)

        if 0 <= ui < hm_w and 0 <= vi < hm_h:
            hm[vi, ui] += 1.0

def save_heatmaps(heatmaps, posters):
    posters_by_name = {p["name"]: p for p in posters}

    for name, hm in heatmaps.items():
        poster_img = posters_by_name[name]["img"]
        poster_h, poster_w = poster_img.shape[:2]

        if hm.max() > 0:
            hm_blur = cv2.GaussianBlur(hm, GAUSSIAN_BLUR_KSIZE, 0)
            hm_norm = (hm_blur / hm_blur.max() * 255).astype(np.uint8)
        else:
            hm_norm = hm.astype(np.uint8)

        hm_color = cv2.applyColorMap(hm_norm, cv2.COLORMAP_JET)
        hm_resized = cv2.resize(hm_color, (poster_w, poster_h), interpolation=cv2.INTER_LINEAR)

        overlay = cv2.addWeighted(poster_img, 0.6, hm_resized, 0.4, 0)
        out_path = f"Sujet4_heatmap_{name}_V3.png"
        cv2.imwrite(out_path, overlay)
        print(f"✅ Heatmap sauvegardée pour {name} : {out_path}")


# ============================================================
# MAIN : PARCOURS VIDÉO + HEATMAPS (PAS DE VIDÉO EN SORTIE)
# ============================================================

def main():
    merged = build_merged_gaze_world()

    orb = cv2.ORB_create(nfeatures=ORB_NFEATURES)
    bf  = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)

    posters = load_posters(orb, POSTERS_DIR)
    if not posters:
        raise RuntimeError("Aucune affiche chargée, vérifie POSTERS_DIR.")
    heatmaps = init_heatmaps(posters)

    cap = cv2.VideoCapture(VIDEO_PATH)
    if not cap.isOpened():
        raise FileNotFoundError(f"Impossible d'ouvrir la vidéo : {VIDEO_PATH}")

    frame_idx = 0
    print("Début du traitement des frames (heatmaps uniquement)...")

    while True:
        ret, frame = cap.read()
        if not ret or frame_idx >= len(merged):
            break

        detections = detect_posters_in_frame(frame, posters, orb, bf)

        row = merged.iloc[frame_idx]
        x = row["gaze x [px]"]
        y = row["gaze y [px]"]

        if not (np.isnan(x) or np.isnan(y)):
            h, w = frame.shape[:2]
            gx = float(np.clip(x, 0, w - 1))
            gy = float(np.clip(y, 0, h - 1))
            update_heatmaps(detections, gx, gy, heatmaps)

        frame_idx += 1
        if frame_idx % 50 == 0:
            print(f"{frame_idx} frames traitées...")

    cap.release()
    print("Fin du parcours vidéo, génération des heatmaps…")
    save_heatmaps(heatmaps, posters)


if __name__ == "__main__":
    main()


K =\n [[884.9203   0.     797.3327]
 [  0.     884.8444 590.4394]
 [  0.       0.       1.    ]]
dist (8) = [-0.1284729   0.10799497 -0.00076467  0.00041167  0.00231511  0.1688897
  0.05152452  0.02927154]
dist (opencv 5) = [-0.1284729   0.10799497 -0.00076467  0.00041167  0.00231511]
Affiche chargée : Anning (2000 keypoints)
Affiche chargée : Barres (2000 keypoints)
Affiche chargée : Bell (2000 keypoints)
Affiche chargée : Bunten-Berry (2000 keypoints)
Affiche chargée : Franklin (2000 keypoints)
Affiche chargée : Gautier (2000 keypoints)
Affiche chargée : Johnson (2000 keypoints)
Affiche chargée : Noether (2000 keypoints)
Début du traitement des frames (heatmaps uniquement)...
50 frames traitées...
100 frames traitées...
150 frames traitées...
200 frames traitées...
250 frames traitées...
300 frames traitées...
350 frames traitées...
400 frames traitées...
450 frames traitées...
500 frames traitées...
550 frames traitées...
600 frames traitées...
650 frames traitées...
700 frames trai